In [ ]:
"""
1. input files
2. load data in that line
3. save data


"""

# 1 xspec

In [5]:
import numpy as np
from glob import glob
import re
import pandas as pd

In [ ]:
def tidy_dict2df(savepath, regname, mdl, bigkeys = ['T', 'Z', 'n']):
    # initialize dict
    output_dict = {'ID':[], 'cstat':[], 'dof':[]}
    for name in bigkeys:
        output_dict[f'{name}-errlo'] = []
        output_dict[f'{name}-errhi'] = []
        output_dict[f'{name}-value'] = []
        output_dict[f'{name}-value'] = []
    

    cts = 0
    dirs = glob(f'{savepath}/SDSS*')
    for dir in dirs:
        srcname = dir.split('/')[-1]
        val_file = f'{dir}/fit_231115/logs/annu_{regname}_{mdl}_freepar.log'
        err_file = f'{dir}/fit_231115/logs/annu_{regname}_{mdl}_chain1000_par.log'
        cst_file = f'{dir}/fit_231115/logs/annu_{regname}_{mdl}_fit.log'

        # read mcmc errors
        if glob(val_file) and glob(err_file):
            output_dict[f'ID'].append(f'{srcname}')
            # read the annuli from eckert regionfile
            with open(err_file) as f:
                lines = f.readlines()
            for i in range(len(bigkeys)):
                if len(lines)> 20:
                    errlo = 999
                    errhi = 999
                else:
                    start_idx = next((index for index, line in enumerate(lines) if 'Parameter' in line), None)
                    
                    errlo = lines[int(start_idx+1+i)].split('(')[-1].split(',')[0]
                    errhi = lines[int(start_idx+1+i)].split('(')[-1].split(',')[-1][:-2]
                output_dict[f'{bigkeys[i]}-errlo'].append(abs(float(errlo)))
                output_dict[f'{bigkeys[i]}-errhi'].append(float(errhi))

            # read value
            with open(val_file) as f:
                text = f.read()
            pattern = r'([+-]?[\d]*\.?[\d]+(?:[eE][-+]?\d+)?)\s+\+/-'
            values = re.findall(pattern, text)
            for i in range(len(bigkeys)):
                output_dict[f'{bigkeys[i]}-value'].append(float(values[i]))
            
            # read cstat
            with open(cst_file) as f:
                lines = f.readlines()[-30:]
            for line in lines:
                if 'Total fit statistic' in line:
                    cstat = float(line.split('tic')[1].split('with')[0].strip())
                    dof = int(line.split('with')[-1].split('d.o.f')[0].strip())
                    output_dict['cstat'].append(cstat)
                    output_dict['dof'].append(dof)
        else:
            cts+=1
            continue
    print(cts)
    return output_dict

1.1 GDEM

In [ ]:
my_rootdir = f'/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP'

new_dict = tidy_dict2df(my_rootdir, 'R500-01', 'GDEM',  bigkeys = ['T', 'Tsig', 'Z', 'n'])
new_df = pd.DataFrame(new_dict)
new_df.to_csv(f'{my_rootdir}/../dats/R500-01_GDEM_xspec.csv')

1.2 1T

In [ ]:
my_rootdir = f'/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP'

new_dict = tidy_dict2df(my_rootdir, 'R500-01', '1T',  bigkeys = ['T', 'Z', 'n'])
new_df = pd.DataFrame(new_dict)
new_df.to_csv(f'{my_rootdir}/../dats/R500-01_1T_xspec.csv')

# 2 spex

In [20]:
def tidy_dict2df(savepath, regname, mdl, bigkeys = ['n', 'T', 'Z']):
    cts = 0
    dirs = glob(f'{savepath}/SDSSTG*')
    print(len(dirs))
    # initialize dict
    output_dict = {'ID':[], 'cstat':[], 'dof':[]}
    
    for name in bigkeys:
        output_dict[f'{name}-errlo'] = np.zeros(len(dirs))
        output_dict[f'{name}-errhi'] = np.zeros(len(dirs))
        output_dict[f'{name}-value'] = np.zeros(len(dirs))
        output_dict[f'cstat'] = np.zeros(len(dirs))
        output_dict[f'dof'] = np.zeros(len(dirs))
        output_dict[f'ID'] = []
    
    for i, dir in enumerate(dirs):
        srcname = dir.split('/')[-1]
        output_dict[f'ID'].append(f'{srcname}')
        err_file = f'{dir}/fit_spex_231115/annu-{regname}-{mdl}_error.out'
        cst_file = f'{dir}/fit_spex_231115/annu-{regname}-{mdl}.out'
        
        if glob(err_file):
            print(err_file)
            # read mcmc errors
            with open(err_file) as f:
                lines = f.readlines()
            if len(lines)> 200:
                errlo = 999
                errhi = 999
            else:
                j = 0
                for line in lines:
                    if 'Errors:' in line:
                        errlo = line.split('Errors:')[-1].split(',')[0]
                        errhi = line.split('Errors:')[-1].split(',')[-1]
                        value = line.split('Errors:')[0].split(':')[-1]
                        # print(value, errlo, errhi)
                        output_dict[f'{bigkeys[j]}-value'][i] = float(value)
                        output_dict[f'{bigkeys[j]}-errlo'][i] = abs(float(errlo))
                        output_dict[f'{bigkeys[j]}-errhi'][i] = float(errhi)
                        j += 1
                        print(errlo, errhi, value)
                        
            # read cstat
            with open(cst_file) as f:
                lines = f.readlines()[-30:]
            for line in lines:
                pattern = re.compile(r"C-statistic\s+:\s+\d+")
                # Find all matches in the input text
                matches = re.findall(pattern, line)
                # Print the matching lines
                for match in matches:
                    # print(match)
                    cstat = float(match.split(':')[1].strip())
                if 'Degrees of freedom' in line:
                    dof = line.split(':')[1].strip()
                    
            output_dict['cstat'][i] = cstat
            output_dict['dof'][i] = dof
    
        else:
            cts += 1
            continue


    print(cstat, cts, )
    return output_dict

2.1 GDEM

In [21]:
my_rootdir = f'/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP'

new_dict = tidy_dict2df(my_rootdir, 'R500-01', 'GDEM',  bigkeys = ['n', 'T', 'Tsig', 'Z'])

# for k, v in new_dict.items():
#     print(k, v)
new_df = pd.DataFrame(new_dict)
print(new_df)
new_df.to_csv(f'{my_rootdir}/../dats/R500-01_GDEM_spex.csv')

48
/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP/SDSSTG40241/fit_spex_231115/annu-R500-01-GDEM_error.out
  -4.61741E+05     4.79384E+05
    4.17685E+06 
  -1.59168E-02     3.62656E-02
    0.91436     
    0.0000         0.11334
     0.0000     
  -3.92592E-02     0.13425
    0.24969     
/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP/SDSSTG22635/fit_spex_231115/annu-R500-01-GDEM_error.out
  -1.67204E+06     5.33954E+05
    3.50672E+06 
  -0.43477          1.7570
     2.5638     
  -0.66947         0.53934
    0.86079     
  -0.10153         0.27125
    0.21829     
/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP/SDSSTG5742/fit_spex_231115/annu-R500-01-GDEM_error.out
/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP/SDSSTG15641/fit_spex_231115/annu-R500-01-GDEM_error.out
  -1.65018E+05     2.06506E+05
    1.14869E+06 
  -0.11370         0.13517
     1.2221     
  -0.12612         0.13651
    0.40186     
/Users/eusracenorth/Documents/work/XGAP-AB

2.2 1T

In [16]:
my_rootdir = f'/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP'

new_dict = tidy_dict2df(my_rootdir, 'R500-01', '1T',  bigkeys = ['n', 'T', 'Z'])
for k, v in new_dict.items():
    print(k, len(v))
new_df = pd.DataFrame(new_dict)
print(new_df)
new_df.to_csv(f'{my_rootdir}/../dats/R500-01_1T_spex.csv')

48
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
16
ID 48
cstat 48
dof 48
n-errlo 48
n-errhi 48
n-value 48
T-errlo 48
T-errhi 48
T-value 48
Z-errlo 48
Z-errhi 48
Z-value 48
             ID   cstat    dof   n-errlo   n-errhi    n-value   T-errlo  \
0   SDSSTG40241   223.0  196.0  363506.0  374062.0  3263240.0  0.015310   
1   SDSSTG22635     0.0    0.0       0.0       0.0        0.0  0.000000   
2    SDSSTG5742   327.0  252.0  380540.0  398119.0  2813710.0  0.012009   
3   SDSSTG15641   379.0  355.0  121673.0  131015.0  1660850.0  0.037469   
4    SDSSTG9771   542.0  199.0  260430.0  274566.0  2406260.0  0.029965   
5    SDSSTG6159     0.0    0.0       0.0       0.0        0.0  0.000000   
6    SDSSTG9370     0.0    0.0       0.0       0.0        0.0  0.000000   
7    SDSSTG4654   805.0  683.0   85443.0   83308.0  1913390.0  0.006312   
8    SDSSTG1601     0.0    0.0       0.0       0.0        0.0  0